### Running the model

In [1]:
import pytorch_lightning as pl
import os
import torch

from mutag.src.datamodules import MUTAGDataModule
from mutag.src.models import GraphLevelGNN
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
torch.manual_seed(42)

In [3]:
datamodule = MUTAGDataModule('./data', 64)
datamodule

In [4]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [5]:
pl.seed_everything(42)

root_dir = os.path.join('./gnnmodel', 'GraphLevel' + 'GraphConv')
os.makedirs(root_dir, exist_ok=True)

Global seed set to 42


In [6]:
trainer = pl.Trainer(
    default_root_dir=root_dir,
    callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc')],
    gpus=1 if str(device).startswith('cuda') else 0,
    max_epochs=500,
    progress_bar_refresh_rate=0
)
trainer.logger._default_hp_metric = None

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
pretrained_filename = os.path.join('./gnnmodel', 'GraphLevel%s.ckpt' % 'GraphConv')
pretrained_filename

'./gnnmodel\\GraphLevelGraphConv.ckpt'

In [8]:
if os.path.isfile(pretrained_filename):
    model = GraphLevelGNN.load_from_checkpoint(pretrained_filename)
else:
    pl.seed_everything(42)
    model = GraphLevelGNN(
        c_in=7,
        c_out=1,
        c_hidden=256,
        num_layers=3,
        layer_name='GraphConv',
        dp_rate=0.0,
        dp_rate_linear=0.5
    )
    trainer.fit(model, datamodule=datamodule)

Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | model       | GraphGNNModel     | 266 K 
1 | loss_module | BCEWithLogitsLoss | 0     
--------------------------------------------------
266 K     Trainable params
0         Non-trainable params
266 K     Total params
1.067     Total estimated model params size (MB)
C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers whic

In [9]:
train_result = trainer.test(model, test_dataloaders=datamodule.train_dataloader(), verbose=False)
train_result

C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:679: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:372: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  rank_zero_warn(
C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


[{'test_acc': 0.912405252456665}]

In [10]:
test_result = trainer.test(model, test_dataloaders=datamodule.test_dataloader(), verbose=False)
test_result

C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\Gilberto\Anaconda3\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


[{'test_acc': 0.8421052694320679}]